<a href="https://colab.research.google.com/github/Ramyaanbazhagan/Ramyaanbazhagan/blob/main/Training%20a%20Hebbian%20Neural%20Network%20architecture%20with%20a%20finance%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

# Step 1: Define activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Step 2: Define loss function
def loss_function(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Step 3: Implement Hebbian Neural Network class
class HebbianNN:
    def __init__(self, input_size, output_size, learning_rate=0.01):
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.learning_rate = learning_rate

    def forward(self, X):
        self.input = X
        self.output = sigmoid(np.dot(X, self.weights))
        return self.output

    def hebbian_update(self):
        self.weights += self.learning_rate * np.dot(self.input.T, self.output)

    def train(self, X, y, epochs=100):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = loss_function(y, y_pred)
            self.hebbian_update()
            print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

# Step 4: Load and preprocess the historical stock prices of Apple (AAPL)
data = yf.download('AAPL', start='2010-01-01', end='2022-02-26')
X = data[['Open', 'High', 'Low', 'Volume']]
y = data['Close'].values.reshape(-1, 1)

# Scale the data using Min-Max Scaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

# Step 5: Train the Hebbian Neural Network
nn = HebbianNN(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1], learning_rate=0.01)
nn.train(X_scaled, y_scaled, epochs=10)

[*********************100%***********************]  1 of 1 completed

Epoch 1, Loss: 0.1387
Epoch 2, Loss: 0.3866
Epoch 3, Loss: 0.5950
Epoch 4, Loss: 0.6561
Epoch 5, Loss: 0.6729
Epoch 6, Loss: 0.6780
Epoch 7, Loss: 0.6796
Epoch 8, Loss: 0.6802
Epoch 9, Loss: 0.6804
Epoch 10, Loss: 0.6805
